In [48]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
df = pd.read_csv('~/Old_Projects/Alerting_Bot/spend_data.csv')

In [4]:
df['business_dt'] = pd.to_datetime(df.business_dt)

In [21]:
def metrics(df: pd.DataFrame) -> pd.DataFrame :
    df['CPM'] = (df.spend_usd / df.impressions * 1000).round(2)
    df['CPI'] = (df.spend_usd / df.installs).round(2)
    df['CPC'] = (df.spend_usd / df.clicks).round(2)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)
    return df

In [22]:
df = df[df.offer == 'yaeda_restaurant']

In [23]:
df = df.groupby(['business_dt', 'offer', 'network']).sum(numeric_only=True).reset_index()

In [24]:
df = metrics(df)

In [25]:
df = df[(df.CPM + df.CPM + df.CPC) > 0]

In [51]:
X = df

In [52]:
X = pd.get_dummies(X)

In [131]:
model = IsolationForest(n_estimators=1000, contamination=0.07, random_state=42)
model.fit(X)

IsolationForest(contamination=0.07, n_estimators=1000, random_state=42)

In [132]:
outliers = model.predict(X)

In [133]:
df['is_anomaly'] = outliers

In [134]:
df[df.is_anomaly == -1]

,business_dt,offer,network,spend_usd,impressions,clicks,installs,CPM,CPI,CPC,is_anomaly
61,2024-09-09,yaeda_restaurant,XapAds,1123.324996,0.0,143,510.0,0.00,2.20,7.86,-1
68,2024-09-10,yaeda_restaurant,XapAds,1577.210289,0.0,319,709.0,0.00,2.22,4.94,-1
96,2024-09-14,yaeda_restaurant,XapAds,2031.555934,0.0,650,918.0,0.00,2.21,3.13,-1
103,2024-09-15,yaeda_restaurant,XapAds,1925.017295,0.0,504,866.0,0.00,2.22,3.82,-1
144,2024-09-21,yaeda_restaurant,VKADS,909.743523,659002.0,838,15.0,1.38,60.65,1.09,-1
186,2024-09-27,yaeda_restaurant,VKADS,1869.380340,1346687.0,1655,46.0,1.39,40.64,1.13,-1
187,2024-09-27,yaeda_restaurant,XapAds,1867.385591,0.0,296,749.0,0.00,2.49,6.31,-1
193,2024-09-28,yaeda_restaurant,VKADS,2322.050522,1575721.0,2107,54.0,1.47,43.00,1.10,-1
194,2024-09-28,yaeda_restaurant,XapAds,1915.284135,0.0,330,768.0,0.00,2.49,5.80,-1
196,2024-09-29,yaeda_restaurant,Bigo,916.434100,1213414.0,10795,0.0,0.76,0.00,0.08,-1
